In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
install.packages("dplyr") 
install.packages("stats")
install.packages("randomForest",dependencies=TRUE)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.0.7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 852103 bytes (832 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write 

In [4]:
%%R
library(stats)
library(dplyr)
library(randomForest)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: randomForest 4.6-14

R[write to console]: Type rfNews() to see new features/changes/bug fixes.

R[write to console]: 
Attaching package: ‘randomForest’


R[write to console]: The following object is masked from ‘package:dplyr’:

    combine




In [5]:
%%R
mydata = read.csv('/content/gdrive/My Drive/DS_CP/diabetes.csv')
str(mydata)

'data.frame':	768 obs. of  9 variables:
 $ Pregnancies             : int  6 1 8 1 0 5 3 10 2 8 ...
 $ Glucose                 : int  148 85 183 89 137 116 78 115 197 125 ...
 $ BloodPressure           : int  72 66 64 66 40 74 50 0 70 96 ...
 $ SkinThickness           : int  35 29 0 23 35 0 32 0 45 0 ...
 $ Insulin                 : int  0 0 0 94 168 0 88 0 543 0 ...
 $ BMI                     : num  33.6 26.6 23.3 28.1 43.1 25.6 31 35.3 30.5 0 ...
 $ DiabetesPedigreeFunction: num  0.627 0.351 0.672 0.167 2.288 ...
 $ Age                     : int  50 31 32 21 33 30 26 29 53 54 ...
 $ Outcome                 : int  1 0 1 0 1 0 1 0 1 1 ...


In [6]:
%%R
index<-sample(2,nrow(mydata),replace=TRUE, prob = c(0.7,0.3))

#training dat a. 
set.seed(123)
training <-mydata[index==1,]
testing<-mydata[index==2,]


In [7]:
%%R
rfm = randomForest(as.factor(Outcome)~.,data=training)

Outcome_pred <-predict(rfm,testing)
testing$Outcome_pred=Outcome_pred

In [8]:
%%R
cfm<-table(testing$Outcome,testing$Outcome_pred)
cfm

   
      0   1
  0 134  25
  1  32  46


In [9]:
%%R
classification_accuracy = sum(diag(cfm))/sum(cfm)
print("Accuracy of the model is:")
classification_accuracy

[1] "Accuracy of the model is:"
[1] 0.7594937
